In [ ]:
# --- 🌟 LLM Explainer Script: Summarize Medical Insight for a Patient ---

import requests
import json
import os

# === 1. File paths ===
base_dir = os.path.dirname(os.getcwd())
abnormal_path = os.path.join(base_dir, "data", "patients", "abnormal_markers_patient_1.txt")
rules_path = os.path.join(base_dir, "data", "patients", "matched_rules_for_patient_1.txt")
output_path = os.path.join(base_dir, "data", "patients", "llm_patient_1_summary.txt")

# === 2. Load patient markers ===
with open(abnormal_path, "r", encoding="utf-8") as f:
    abnormal = f.read()

# === 3. Load matched rules ===
with open(rules_path, "r", encoding="utf-8") as f:
    all_rules = f.readlines()
    top_100_rules = all_rules[:100]

# === 4. Prepare the prompt ===
# Convert the rules list into a single block of text
rules_block = "\n".join(rule.strip() for rule in top_100_rules)

# prompt = f"""
# You are a medical assistant AI.

# Below are association rules generated from a patient's abnormal lab results using the Apriori algorithm. Your job is to analyze them **as a whole**, not individually.

# 📦 Rules:
# {rules_block}

# Now, based on these rules, please provide:

# 1. 🧠 **What is the patient likely suffering from?** (Brief explanation of possible conditions.)
# 2. 🧪 **What 1–2 follow-up medical tests** should the doctor consider?
# 3. 🩺 **What 1-2 medical advice** should be given to the patient?

# Return the answer in formal, simple and clear medical language. Do **not** explain each rule separately — just summarize the insights as one concise diagnosis.

#""""


prompt = f"""
You are a medical assistant AI.

You are given:
- A list of **abnormal medical lab markers** for a patient.
- A list of **association rules** (patterns found in similar patients) related to these markers.

🎯 Your task:
Generate a clear medical summary for the doctor based on these inputs.

1. **What abnormal markers were found?**
For each marker, write:
- What the marker is
- What the abnormal value (↑ or ↓) might indicate medically (e.g., liver damage, kidney issue)

2. **What is the most likely medical issue (diagnosis)?**
Summarize based on the patterns found in the rules and the abnormal markers.

3. **What 1–2 follow-up medical tests** should the doctor consider to confirm the issue?

4. **What 1–2 medical recommendations** should be given to the patient?

📌 Rules to follow: 
- Be short, clear, and medically informative.
- DO NOT repeat all rules. Focus on summarizing the patterns.
- Be formal and clinical in tone.
- Avoid making absolute claims (say “may suggest”, “likely”, etc.)

Example input:
Abnormal markers: [lbxscr↓, lbxsgb↑, lbdscrsi↓]  
Matching rules: 
- If a patient has [lbxsgb↑, lbxscr↓], they likely also have [lbdscrsi↓, lbdsgbsi↑]...

Now generate your medical summary.
"""


# === 5. Call the LLM ===
API_KEY = "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"
headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

data = {
    "model": "meta-llama/llama-3.3-8b-instruct:free",
    "messages": [
        {"role": "system", "content": "You are a helpful medical assistant."},
        {"role": "user", "content": prompt}
    ]
}

try:
    print("🔄 Sending to LLM...")
    response = requests.post("https://openrouter.ai/api/v1/chat/completions", headers=headers, data=json.dumps(data))
    response.raise_for_status()
    reply = response.json()["choices"][0]["message"]["content"]

    with open(output_path, "w", encoding="utf-8") as out:
        out.write(reply)

    print("✅ Summary saved to:", output_path)
    print("\n🩺 LLM Response:\n")
    print(reply)

except Exception as e:
    print("❌ Error:", e)


🔄 Sending to LLM...
❌ Error: 404 Client Error: Not Found for url: https://openrouter.ai/api/v1/chat/completions
